In [8]:
%load_ext autoreload
%autoreload 2
from llm_utils import limp_llm_plan, build_intent_dict

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
## Test instructions:
"Pick up the book on the table with the red cover."
"Place the vase near the flowers on the table."
"Place the coke can beside the pringles on the counter."
"Bring the book and the magazine on the nightstand."
"Bring the mug from the table near the sink."

In [2]:
## Test example
# Transcript: Place the Coke can beside the Pringles on the counter.

# interpretation_1
# "Place the coke can (which is currently beside the pringles) on some counter."
intent_labels_1 = {"place": 0, "the": 0, "coke": 1, "can": 1, "beside": 0, "the": 0, "pringles": 2, "on": 0, "the": 0, "counter": 1}


# interpretation_2
# "Take the coke can (which is somewhere else) and place it beside the pringles (that are already on the counter).",
intent_labels_2 = {"place": 0,"the": 0,"coke": 1,"can": 1,"beside": 0,"the": 0,"pringles": 1,"on": 0,"the": 0,"counter": 2}


In [3]:
instruction = input("Enter instruction: ")

In [6]:
passedin_intent_dict = {'Goal_intent': ['coke', 'can', 'counter'],
 'Avoidance_intent': [],
 'Detail_intent': []}

In [12]:
baseline_prompt_path = "prompts/baseline_asr_prompt.txt"
# baseline_prompt_path = "prompts/prosody_intent_prompt.txt"

# passedin_intent_dict = build_intent_dict(intent_labels_2)
limp_llmplaner_incontext_prompt = open(baseline_prompt_path, "r").read() 
limp_planner_output = limp_llm_plan(limp_llmplaner_incontext_prompt,instruction, passedin_intent_dict, verbose=False)
print("**************************************************\nInstruction:",instruction)
print(f"--------------------------------------------------\nLLM Planner Output\n--------------------------------------------------\n{limp_planner_output}\n**************************************************")

		Model: gpt-4o || Deterministic?: True
**************************************************
Instruction: Place the Coke can beside the Pringles on the counter
--------------------------------------------------
LLM Planner Output
--------------------------------------------------
1. near[coke_can]
2. pick[coke_can]
3. near[counter]
4. release[coke_can,pringles::isnextto(counter)]
**************************************************
